# OpenPose

OpenPose: https://github.com/CMU-Perceptual-Computing-Lab/openpose.git

Terminology

* output : output of OpenPose, which is a JSON file containing the keypoints of a frame in a video (1 frame : 1 JSON;  1 video : n frame)
* output_path : Google drive path to this output JSON file 
* keypoint_coordinate : coordinates of keypoints extracted from output. Each keypoint has an x-coordinate, y-coordinate, and c-confidence score.
* pose : mapping between body part and its keypoint coordinate (e.g. Nose: x, y, c)





# 1 Extract JSON from datasets using OpenPose

## [=====SKIP=====] 1.1 Install OpenPose

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  

## 1.2 Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## [=====SKIP=====] 1.3 Run OpenPose on UR Fall

In [ ]:
# Set variables
ur_dataset_len = 70
ur_video_folder_path = '/content/gdrive/MyDrive/Dataset_UR_Fall/'
ur_output_folder_path = '/content/gdrive/MyDrive/JSON_UR_Fall/output/'

In [ ]:
# Run OpenPose on each video in dataset
import os.path

for i in range(0, ur_dataset_len):
  print(i)
  video_path = ur_video_folder_path + str(i) +'.mp4'
  !cd openpose && ./build/examples/openpose/openpose.bin --video {video_path} --write_json ./output/ --display 0  --write_video ../openpose.avi

In [ ]:
# Copy output to Google Drive
%cp -av "/content/openpose/output" "/content/gdrive/MyDrive/JSON_UR_Fall"

## [=====SKIP=====] 1.4 Run OpenPose on Charfi

In [ ]:
# Set variables
charfi_dataset_len = 221
charfi_video_folder_path = '/content/gdrive/MyDrive/Dataset_Charfi/'
charfi_output_folder_path = '/content/gdrive/MyDrive/JSON_Charfi/'

In [ ]:
# Run OpenPose on each video in dataset
for i in range(0, charfi_dataset_len):
  print(i)
  video_path = charfi_video_folder_path + str(i) +'.mp4'
  !cd openpose && ./build/examples/openpose/openpose.bin --video {video_path} --write_json ./output/ --display 0  --write_video ../openpose.avi

In [ ]:
# Copy output to Google Drive
%cp -av "/content/openpose/output" "/content/gdrive/MyDrive/JSON_Charfi"

In [ ]:
# Move JSON files from common Google Drive folder to seperate folder per video
import os

for i in range (0, charfi_dataset_len):
  print(f'Processing video {i}')
  destination = f'/content/gdrive/MyDrive/JSON_Charfi/Charfi_{i}'
  os.mkdir(destination)

  for j in range (2000):
    frame_num = str(j).zfill(12)
    file_name = f'/content/gdrive/MyDrive/JSON_Charfi/output/{i}_{frame_num}_keypoints.json'
    try:
      open(file_name)
    except: 
      continue
    %mv {file_name} {destination}

# 2 Define functions

## 2.1 Construct JSON file path (output of OpenPose)

In [ ]:
def charfi_get_env(vid_num: int):
  """Returns environment name that is part of path to JSON file. Only applicable to Charfi dataset.
  Args:
    vid_num: number of the video
  """
  if vid_num < 70: 
    return "0_Coffee_room"
  elif 70 <= vid_num < 130:
      return "1_Home"
  elif 130 <= vid_num < 157:
      return "2_Lecture"
  elif 157 <= vid_num < 189:
      return "3_Office"
  else:
      return "4_Office_2"

In [ ]:
def construct_output_path(dataset:str, vid_num: str, frame_num: str):
  """Returns path to JSON file that is the output of OpenPose
  Args:
    dataset: name of the dataset
    vid_num: number of the video 
    frame_num: number of the frame 
  """
  frame_num_int = str(frame_num).zfill(12)

  if dataset == 'ur':
    return f'{ur_output_folder_path}{vid_num}_{frame_num_int}_keypoints.json'
  elif dataset == 'charfi':
    env = charfi_get_env(vid_num)
    return f'{charfi_output_folder_path}{env}/Charfi_{vid_num}/{vid_num}_{frame_num_int}_keypoints.json'

In [ ]:
#Test 
#print(construct_output_path('ur', 0, 0))
#print(construct_output_path('charfi', 0, 0))

## 2.2 Convert JSON file path to pose information

In [ ]:
# Source: https://cmu-perceptual-computing-lab.github.io/openpose/web/html/doc/md_doc_02_output.html
openpose_pose_keypoints_2d_map = [
                                  [0,  "Nose"],
                                  [1,  "Neck"],
                                  [2,  "RShoulder"],
                                  [3,  "RElbow"],
                                  [4,  "RWrist"],
                                  [5,  "LShoulder"],
                                  [6,  "LElbow"],
                                  [7,  "LWrist"],
                                  [8,  "MidHip"],
                                  [9,  "RHip"],
                                  [10, "RKnee"],
                                  [11, "RAnkle"],
                                  [12, "LHip"],
                                  [13, "LKnee"],
                                  [14, "LAnkle"],
                                  [15, "REye"],
                                  [16, "LEye"],
                                  [17, "REar"],
                                  [18, "LEar"],
                                  [19, "LBigToe"],
                                  [20, "LSmallToe"],
                                  [21, "LHeel"],
                                  [22, "RBigToe"],
                                  [23, "RSmallToe"],
                                  [24, "RHeel"],
                                  [25, "Background"]
                                  ]

In [ ]:
import json 
  
def map_output_path_to_pose(output_path: str):
  """Returns list of {body parts : [x coordinate, y coordinate, confidence score]} dict 
  if output path exists & contains coordinates, 0 otherwise
  Args: 
    output_path: path to the JSON file
  """
  try:
     open(output_path)
  # IF output_path does NOT EXIST
  except: 
    return 0

  output = json.load(open(output_path))

  # IF output_path EXISTS and no coordinate is detected 
  if len(output["people"]) == 0: 
    return 0
  # IF output_path EXISTS and coordinate is detected 
  else: 
    pose = {}
    keypoint_coordinate = output["people"][0]["pose_keypoints_2d"]

    for i in range (25):
      j = i * 3
      pose[openpose_pose_keypoints_2d_map[i][1]] = keypoint_coordinate[j:j+3]

    return pose

In [ ]:
# Test 
# UR dataset video 0 frame 5 
#ur_0_5_path = construct_output_path('ur', 0, 5)
#ur_0_5_pose = map_output_path_to_pose(ur_0_5_path)
#print(f'UR Vid 0 Frame 5: \n {ur_0_5_pose}')

# Charfi dataset video 1 frame 5
#charfi_1_5_path = construct_output_path('charfi', 1, 5)
#charfi_1_5_pose = map_output_path_to_pose(charfi_1_5_path)
#print(f'Charfi Vid 1 Frame 5: \n {charfi_1_5_pose}')

## 2.3 Detect fall based on pose information


In [ ]:
def check_velocity(
    left_hip_y_t1: float, 
    right_hip_y_t1: float, 
    left_hip_y_t2: float, 
    right_hip_y_t2: float, 
    interval: float):
  """Returns True if velocity exceeds threshold (0.009), False otherwise
  Args: 
    left_hip_y_t1: y coordinate of left hip at time 1
    right_hip_y_t1: y coordinate of right hip at time 1
    left_hip_y_t2: y coordinate of left hip at time 2
    right_hip_y_t2: y coordinate of right hip at time 2
    interval: difference (in second) between time 1 and time 2
  """
  center_hip_y_t1 = (left_hip_y_t1 + right_hip_y_t1) / 2
  center_hip_y_t2 = (left_hip_y_t2 + right_hip_y_t2) / 2
  velocity = abs(center_hip_y_t1 - center_hip_y_t2) * 0.0002645833 / interval
  
  # 220613
  #print(round(velocity, 5))
  
  return velocity > 0.009

In [ ]:
import math
def check_angle(
      left_ankle_x: float,
      left_ankle_y: float,
      right_ankle_x: float,
      right_ankle_y: float,
      head_x: float,
      head_y: float):
  """Returns True if angle a exceeds threshold (45 degree), False otherwise 
              head
                  (`
                  ( a `
      left_ankle-------x----- right_ankle

  Point x: Middle point between left and right ankle
  Angle a: Acute angle between line connecting head & point x, and ground
  Args:
    left_ankle_x: x coordinate of left ankle
    left_ankle_y: y coordinate of left ankle
    right_ankle_x: x coordinate of right ankle
    right_ankle_y: y coordinate of right ankle
    head_x: x coordinate of head
    head_y: y coordinate of head
  """
  middle_ankle_x = (left_ankle_x + right_ankle_x) / 2
  middle_ankle_y = (left_ankle_y + right_ankle_y) / 2
  if head_x == middle_ankle_x: 
    return False
  else:
    angle_in_radian = math.atan(abs(head_y - middle_ankle_y)/(head_x - middle_ankle_x))
    angle_in_degree = abs(angle_in_radian * 180 / math.pi)

    # 220613
    #print(round(angle_in_degree, 5))

    return angle_in_degree < 45

In [ ]:
def check_ratio(all_x: list, all_y: list):
  """Returns True if ratio exceeds threshold, False otherwise
  Args: 
      all_x: list of all x coordinates extracted from frame
      all_y: list of all y coordinates extracted from frame
  """
  min_x_above_0 = min(i for i in all_x if i > 0)
  min_y_above_0 = min(i for i in all_y if i > 0)

  width = max(all_x) - min_x_above_0
  height = max(all_y) - min_y_above_0
  ratio = width / height

  # 220613
  #print(round(ratio, 5))

  return ratio > 1

In [ ]:
def openpose_detect_fall_in_pose(dataset, pose_frame_t1, pose_frame_t2):
  """Returns True if Fall is detected in the pose of frame t2, False otherwise
  Args: 
    dataset: name of the dataset
    pose_frame_t1: list of {body parts : [x coordinate, y coordinate, confidence score]} dicts of frame at time 1
    pose_frame_t2: list of {body parts : [x coordinate, y coordinate, confidence score]} dicts of frame at time 2
  """
  fall = False
  interval = (1/30*5) if dataset == 'ur' else (1/24*5)

  # map this function's param to check_velocity function param
  left_hip_y_t1 = pose_frame_t1["LHip"][1]
  right_hip_y_t1 = pose_frame_t1["RHip"][1]
  left_hip_y_t2 = pose_frame_t2["LHip"][1]
  right_hip_y_t2 = pose_frame_t2["RHip"][1]

  # map this function's param to check_angle function param
  left_ankle_x = pose_frame_t2["LAnkle"][0]
  left_ankle_y = pose_frame_t2["LAnkle"][1]
  right_ankle_x = pose_frame_t2["RAnkle"][0]
  right_ankle_y = pose_frame_t2["RAnkle"][1]
  head_x = pose_frame_t2["Nose"][0]
  head_y = pose_frame_t2["Nose"][1]

  # map this function's param to check_ratio function param
  all_x = []
  all_y = []
  for item in pose_frame_t2.values():
    all_x.append(item[0])
    all_y.append(item[1])

  condition_1 = check_velocity(left_hip_y_t1, right_hip_y_t1, left_hip_y_t2, right_hip_y_t2, interval)
  condition_2 = check_angle(left_ankle_x, left_ankle_y, right_ankle_x, right_ankle_y, head_x, head_y)
  condition_3 = check_ratio(all_x, all_y)

  # 220613
  #print(f'Condition 1: {condition_1}, Condition 2: {condition_2}, Condition 3: {condition_3}')

  if condition_1 & condition_2 & condition_3:
    fall = True
  return fall

In [ ]:
# Test
#charfi_1_0_path = construct_output_path('charfi', 1, 0)
#charfi_1_0_pose = map_output_path_to_pose(charfi_1_0_path)

#charfi_1_5_path = construct_output_path('charfi', 1, 5)
#charfi_1_5_pose = map_output_path_to_pose(charfi_1_5_path)

#charfi_1_5_fall = openpose_detect_fall_in_pose('charfi', charfi_1_0_pose, charfi_1_5_pose)
#print(charfi_1_5_fall)

## 2.4 Detect fall in a frame


In [ ]:
def openpose_detect_fall_in_frame(
    dataset: str,
    vid_num: int, 
    frame_num: int):
  """
  Returns 1 if fall is detected in a frame, 0 otherwise.
  Args: 
    dataset: name of the dataset
    vid_num: number of the video
    frame_num: number of the frame
  """
  # 220613
  #print(f'__Frame: {frame_num}')
  fall = 0

  frame_path = construct_output_path(dataset, vid_num, frame_num)
  frame_prev_5_path = construct_output_path(dataset, vid_num, frame_num-5)

  frame_pose = map_output_path_to_pose(frame_path)
  frame_prev_5_pose = map_output_path_to_pose(frame_prev_5_path)

  if frame_pose == 0:
    # 220613
    #print(f'No pose detected or path does not exist')
    
    return fall
  elif frame_prev_5_pose == 0:
    # 220613
    #print(f'No pose detected or path does not exist for frame {frame_num-5}')
    
    return fall
  elif openpose_detect_fall_in_pose(dataset, frame_pose, frame_prev_5_pose):
    print (f'==========Fall detected in frame {frame_num}==========')
    fall = 1
    return fall

In [ ]:
# Test 
#charfi_1_5_fall = openpose_detect_fall_in_frame('charfi', 1, 5)
#print(charfi_1_5_fall)

## 2.5 Detect fall in a video

In [ ]:
def openpose_detect_fall_in_video(dataset: str, vid_num: int):
  """
  Returns 1 if fall is detected in at least 01 frame of video, 0 otherwise.
  Args: 
    dataset: name of the dataset
    vid_num: number of the video
  """
  fall = 0
  vid_max_length = 450 if dataset == 'ur' else 2000

  for i in range (vid_max_length, 4, -1):
    if openpose_detect_fall_in_frame(dataset, vid_num, i):
      fall = 1
      break
  return fall

In [ ]:
# Test 
#print(openpose_detect_fall_in_video('ur', 0))
#print(openpose_detect_fall_in_video('charfi', 3))

## 2.6 Evaluate

In [ ]:
def evaluate(truth: list, prediction: list):
  """
  Returns number of True Positive, False Negative, True Negative, False Positive
  and calculation of Accuracy, Sensitivity and Specificity
  Args:
    truth: list of label of each video in dataset
    prediction: list of prediction for each video in dataset
  """
  TP = FN = FP = TN = Others = 0
  for i in range (0, 70):
    if (truth[i] == 1) & (prediction[i] == 1): 
      TP += 1
    elif (truth[i] == 0) & (prediction[i] == 1): 
      FP += 1
    elif (truth[i] == 1) & (prediction[i] == 0):
      FN += 1
    elif (truth[i] == 0) & (prediction[i] == 0):
      TN += 1
    else: 
      print(i)
      Others += 1
    
  Accuracy = round((TP + TN) / (TP + FP + TN + FN) * 100, 2)
  Sensitivity = round(TP / (TP + FN) * 100, 2) if (TP + FN) != 0 else "Cannot calcuate due to division by 0"
  Specificity = round(TN / (TN + FP) * 100, 2) if (TN + FP) != 0 else "Cannot calcuate due to division by 0"
    
  print(f'True Positive: {TP} \nFalse Negative: {FN} \nFalse Positive: {FP} \nTrue Negative: {TN} \nOthers: {Others}')
  print(f'Accuracy: {Accuracy}% \nSensitivity: {Sensitivity}% \nSpecificity: {Specificity}%')

# 3 Detect fall in UR Fall dataset

In [ ]:
ur_prediction_0_9 = []
ur_prediction_10_19 = []
ur_prediction_20_29 = []
ur_prediction_30_39 = []
ur_prediction_40_49 = []
ur_prediction_50_59 = []
ur_prediction_60_69 = [] 

In [ ]:
for i in range (0, 10):
  print(f'Video {i}')
  ur_prediction_0_9.append(openpose_detect_fall_in_video('ur', i))

Video 0
==========Fall detected in frame 130==========
Video 1
==========Fall detected in frame 109==========
Video 2
==========Fall detected in frame 187==========
Video 3
==========Fall detected in frame 94==========
Video 4
==========Fall detected in frame 120==========
Video 5
==========Fall detected in frame 63==========
Video 6
==========Fall detected in frame 153==========
Video 7
==========Fall detected in frame 88==========
Video 8
==========Fall detected in frame 179==========
Video 9
==========Fall detected in frame 128==========


In [ ]:
print(ur_prediction_0_9)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
for i in range (10, 20):
  print(f'Video {i}')
  ur_prediction_10_19.append(openpose_detect_fall_in_video('ur', i))

Video 10
==========Fall detected in frame 110==========
Video 11
==========Fall detected in frame 90==========
Video 12
Video 13
==========Fall detected in frame 49==========
Video 14
==========Fall detected in frame 57==========
Video 15
==========Fall detected in frame 49==========
Video 16
Video 17
==========Fall detected in frame 64==========
Video 18
Video 19
==========Fall detected in frame 55==========


In [ ]:
print(ur_prediction_10_19)

[1, 1, 0, 1, 1, 1, 0, 1, 0, 1]


In [ ]:
for i in range (20, 30):
  print(f'Video {i}')
  ur_prediction_20_29.append(openpose_detect_fall_in_video('ur', i))

Video 20
Video 21
==========Fall detected in frame 55==========
Video 22
Video 23
==========Fall detected in frame 41==========
Video 24
==========Fall detected in frame 70==========
Video 25
==========Fall detected in frame 51==========
Video 26
==========Fall detected in frame 77==========
Video 27
==========Fall detected in frame 65==========
Video 28
Video 29
==========Fall detected in frame 48==========


In [ ]:
print(ur_prediction_20_29)

[0, 1, 0, 1, 1, 1, 1, 1, 0, 1]


In [ ]:
for i in range (30, 40):
  print(f'Video {i}')
  ur_prediction_30_39.append(openpose_detect_fall_in_video('ur', i))

Video 30
Video 31
Video 32
Video 33
Video 34
Video 35
Video 36
Video 37
Video 38
Video 39
==========Fall detected in frame 234==========


In [ ]:
print(ur_prediction_30_39)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [ ]:
for i in range (40, 50):
  print(f'Video {i}')
  ur_prediction_40_49.append(openpose_detect_fall_in_video('ur', i))

Video 40
==========Fall detected in frame 299==========
Video 41
==========Fall detected in frame 206==========
Video 42
==========Fall detected in frame 236==========
Video 43
==========Fall detected in frame 9==========
Video 44
==========Fall detected in frame 231==========
Video 45
Video 46
==========Fall detected in frame 161==========
Video 47
==========Fall detected in frame 225==========
Video 48
==========Fall detected in frame 246==========
Video 49
==========Fall detected in frame 234==========


In [ ]:
print(ur_prediction_40_49)

[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]


In [ ]:
for i in range (50, 60):
  print(f'Video {i}')
  ur_prediction_50_59.append(openpose_detect_fall_in_video('ur', i))

Video 50
==========Fall detected in frame 243==========
Video 51
==========Fall detected in frame 178==========
Video 52
==========Fall detected in frame 159==========
Video 53
Video 54
Video 55
Video 56
Video 57
Video 58
Video 59
==========Fall detected in frame 213==========


In [ ]:
print(ur_prediction_50_59)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [ ]:
for i in range (60, 70):
  print(f'Video {i}')
  ur_prediction_60_69.append(openpose_detect_fall_in_video('ur', i))

Video 60
==========Fall detected in frame 247==========
Video 61
==========Fall detected in frame 192==========
Video 62
==========Fall detected in frame 174==========
Video 63
==========Fall detected in frame 175==========
Video 64
==========Fall detected in frame 197==========
Video 65
==========Fall detected in frame 303==========
Video 66
==========Fall detected in frame 322==========
Video 67
Video 68
==========Fall detected in frame 196==========
Video 69
==========Fall detected in frame 268==========


In [ ]:
print(ur_prediction_60_69)

[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]


In [ ]:
ur_prediction = ur_prediction_0_9 + ur_prediction_10_19 + ur_prediction_20_29 + ur_prediction_30_39 + ur_prediction_40_49 + ur_prediction_50_59 + ur_prediction_60_69

In [ ]:
print(ur_prediction)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]


# 4 Detect fall in Charfi dataset

## 4.1 Coffee room

In [ ]:
charfi_prediction_coffee_room_0_9 = []
charfi_prediction_coffee_room_10_19 = []
charfi_prediction_coffee_room_20_29 = []
charfi_prediction_coffee_room_30_39 = []
charfi_prediction_coffee_room_40_49 = []
charfi_prediction_coffee_room_50_59 = []
charfi_prediction_coffee_room_60_69 = []

In [ ]:
for i in range (0, 10):
  print(f'Video {i}')
  charfi_prediction_coffee_room_0_9.append(openpose_detect_fall_in_video('charfi', i))

Video 0
==========Fall detected in frame 64==========
Video 1
Video 2
==========Fall detected in frame 247==========
Video 3
==========Fall detected in frame 148==========
Video 4
==========Fall detected in frame 180==========
Video 5
==========Fall detected in frame 191==========
Video 6
==========Fall detected in frame 163==========
Video 7
Video 8
==========Fall detected in frame 125==========
Video 9
==========Fall detected in frame 226==========


In [ ]:
print(charfi_prediction_coffee_room_0_9)

[1, 0, 1, 1, 1, 1, 1, 0, 1, 1]


In [ ]:
for i in range (10, 20):
  print(f'Video {i}')
  charfi_prediction_coffee_room_10_19.append(openpose_detect_fall_in_video('charfi', i))

Video 10
==========Fall detected in frame 398==========
Video 11
Video 12
Video 13
Video 14
Video 15
==========Fall detected in frame 61==========
Video 16
==========Fall detected in frame 37==========
Video 17
==========Fall detected in frame 172==========
Video 18
==========Fall detected in frame 113==========
Video 19
==========Fall detected in frame 345==========


In [ ]:
print(charfi_prediction_coffee_room_10_19)

[1, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [ ]:
for i in range (20, 30):
  print(f'Video {i}')
  charfi_prediction_coffee_room_20_29.append(openpose_detect_fall_in_video('charfi', i))

Video 20
==========Fall detected in frame 101==========
Video 21
==========Fall detected in frame 148==========
Video 22
==========Fall detected in frame 294==========
Video 23
==========Fall detected in frame 223==========
Video 24
Video 25
Video 26
==========Fall detected in frame 173==========
Video 27
==========Fall detected in frame 76==========
Video 28
Video 29


In [ ]:
print(charfi_prediction_coffee_room_20_29)

[1, 1, 1, 1, 0, 0, 1, 1, 0, 0]


In [ ]:
for i in range (30, 40):
  print(f'Video {i}')
  charfi_prediction_coffee_room_30_39.append(openpose_detect_fall_in_video('charfi', i))

Video 30
Video 31
==========Fall detected in frame 221==========
Video 32
==========Fall detected in frame 105==========
Video 33
Video 34
Video 35
Video 36
Video 37
==========Fall detected in frame 253==========
Video 38
==========Fall detected in frame 81==========
Video 39
==========Fall detected in frame 273==========


In [ ]:
print(charfi_prediction_coffee_room_30_39)

[0, 1, 1, 0, 0, 0, 0, 1, 1, 1]


In [ ]:
for i in range (40, 50):
  print(f'Video {i}')
  charfi_prediction_coffee_room_40_49.append(openpose_detect_fall_in_video('charfi', i))

Video 40
==========Fall detected in frame 377==========
Video 41
==========Fall detected in frame 127==========
Video 42
==========Fall detected in frame 654==========
Video 43
Video 44
==========Fall detected in frame 118==========
Video 45
==========Fall detected in frame 202==========
Video 46
==========Fall detected in frame 657==========
Video 47
==========Fall detected in frame 681==========
Video 48
==========Fall detected in frame 438==========
Video 49


In [ ]:
print(charfi_prediction_coffee_room_40_49)

[1, 1, 1, 0, 1, 1, 1, 1, 1, 0]


In [ ]:
for i in range (50, 60):
  print(f'Video {i}')
  charfi_prediction_coffee_room_50_59.append(openpose_detect_fall_in_video('charfi', i))

Video 50
==========Fall detected in frame 96==========
Video 51
Video 52
Video 53
Video 54
Video 55
Video 56
Video 57
==========Fall detected in frame 203==========
Video 58
Video 59


In [ ]:
print(charfi_prediction_coffee_room_50_59)

[1, 0, 0, 0, 0, 0, 0, 1, 0, 0]


In [ ]:
for i in range (60, 70):
  print(f'Video {i}')
  charfi_prediction_coffee_room_60_69.append(openpose_detect_fall_in_video('charfi', i))

Video 60
Video 61
==========Fall detected in frame 194==========
Video 62
Video 63
==========Fall detected in frame 150==========
Video 64
Video 65
Video 66
Video 67
Video 68
Video 69


In [ ]:
print(charfi_prediction_coffee_room_60_69)

[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]


In [ ]:
charfi_prediction_coffee_room = charfi_prediction_coffee_room_0_9 + charfi_prediction_coffee_room_10_19 + charfi_prediction_coffee_room_20_29 + charfi_prediction_coffee_room_30_39 + charfi_prediction_coffee_room_40_49 + charfi_prediction_coffee_room_50_59 + charfi_prediction_coffee_room_60_69

In [ ]:
print(charfi_prediction_coffee_room)

[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]


## 4.2 Home

In [ ]:
charfi_prediction_home_70_79 = []
charfi_prediction_home_80_89 = []
charfi_prediction_home_90_99 = []
charfi_prediction_home_100_109 = []
charfi_prediction_home_110_119 = []
charfi_prediction_home_120_129 = []

In [ ]:
for i in range (70, 80):
  print(f'Video {i}')
  charfi_prediction_home_70_79.append(openpose_detect_fall_in_video('charfi', i))

Video 70
Video 71
==========Fall detected in frame 232==========
Video 72
==========Fall detected in frame 232==========
Video 73
==========Fall detected in frame 185==========
Video 74
==========Fall detected in frame 122==========
Video 75
==========Fall detected in frame 144==========
Video 76
==========Fall detected in frame 158==========
Video 77
==========Fall detected in frame 143==========
Video 78
==========Fall detected in frame 202==========
Video 79


In [ ]:
print(charfi_prediction_home_70_79)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 0]


In [ ]:
for i in range (80, 90):
  print(f'Video {i}')
  charfi_prediction_home_80_89.append(openpose_detect_fall_in_video('charfi', i))

Video 80
Video 81
==========Fall detected in frame 180==========
Video 82
Video 83
Video 84
Video 85
==========Fall detected in frame 260==========
Video 86
==========Fall detected in frame 180==========
Video 87
==========Fall detected in frame 213==========
Video 88
==========Fall detected in frame 163==========
Video 89
==========Fall detected in frame 196==========


In [ ]:
print(charfi_prediction_home_80_89)

[0, 1, 0, 0, 0, 1, 1, 1, 1, 1]


In [ ]:
for i in range (90, 100):
  print(f'Video {i}')
  charfi_prediction_home_90_99.append(openpose_detect_fall_in_video('charfi', i))

Video 90
==========Fall detected in frame 196==========
Video 91
==========Fall detected in frame 228==========
Video 92
==========Fall detected in frame 172==========
Video 93
==========Fall detected in frame 147==========
Video 94
==========Fall detected in frame 206==========
Video 95
==========Fall detected in frame 143==========
Video 96
==========Fall detected in frame 178==========
Video 97
==========Fall detected in frame 215==========
Video 98
==========Fall detected in frame 257==========
Video 99


In [ ]:
print(charfi_prediction_home_90_99)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]


In [ ]:
for i in range (100, 110):
  print(f'Video {i}')
  charfi_prediction_home_100_109.append(openpose_detect_fall_in_video('charfi', i))

Video 100
==========Fall detected in frame 292==========
Video 101
==========Fall detected in frame 190==========
Video 102
Video 103
==========Fall detected in frame 233==========
Video 104
Video 105
==========Fall detected in frame 130==========
Video 106
Video 107
Video 108
Video 109


In [ ]:
print(charfi_prediction_home_100_109)

[1, 1, 0, 1, 0, 1, 0, 0, 0, 0]


In [ ]:
for i in range (110, 120):
  print(f'Video {i}')
  charfi_prediction_home_110_119.append(openpose_detect_fall_in_video('charfi', i))

Video 110
==========Fall detected in frame 184==========
Video 111
Video 112
Video 113
Video 114
Video 115
==========Fall detected in frame 53==========
Video 116
Video 117
Video 118
Video 119


In [ ]:
print(charfi_prediction_home_110_119)

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [ ]:
for i in range (120, 130):
  print(f'Video {i}')
  charfi_prediction_home_120_129.append(openpose_detect_fall_in_video('charfi', i))

Video 120
Video 121
Video 122
Video 123
Video 124
==========Fall detected in frame 157==========
Video 125
Video 126
==========Fall detected in frame 55==========
Video 127
Video 128
==========Fall detected in frame 189==========
Video 129


In [ ]:
print(charfi_prediction_home_120_129)

[0, 0, 0, 0, 1, 0, 1, 0, 1, 0]


In [ ]:
charfi_prediction_home = charfi_prediction_home_70_79 + charfi_prediction_home_80_89 + charfi_prediction_home_90_99 + charfi_prediction_home_100_109 + charfi_prediction_home_110_119 + charfi_prediction_home_120_129

In [ ]:
print(charfi_prediction_home)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0]


## 4.3 Lecture

In [ ]:
charfi_prediction_lecture_130_139 = []
charfi_prediction_lecture_140_149 = []
charfi_prediction_lecture_150_156 = []

In [ ]:
for i in range (130, 140):
  print(f'Video {i}')
  charfi_prediction_lecture_130_139.append(openpose_detect_fall_in_video('charfi', i))

Video 130
==========Fall detected in frame 200==========
Video 131
==========Fall detected in frame 470==========
Video 132
==========Fall detected in frame 489==========
Video 133
Video 134
==========Fall detected in frame 288==========
Video 135
==========Fall detected in frame 182==========
Video 136
==========Fall detected in frame 252==========
Video 137
==========Fall detected in frame 608==========
Video 138
==========Fall detected in frame 810==========
Video 139


In [ ]:
print(charfi_prediction_lecture_130_139)

[1, 1, 1, 0, 1, 1, 1, 1, 1, 0]


In [ ]:
for i in range (140, 150):
  print(f'Video {i}')
  charfi_prediction_lecture_140_149.append(openpose_detect_fall_in_video('charfi', i))

Video 140
Video 141
Video 142
==========Fall detected in frame 341==========
Video 143
==========Fall detected in frame 77==========
Video 144
Video 145
==========Fall detected in frame 50==========
Video 146
Video 147
Video 148
==========Fall detected in frame 63==========
Video 149


In [ ]:
print(charfi_prediction_lecture_140_149)

[0, 0, 1, 1, 0, 1, 0, 0, 1, 0]


In [ ]:
for i in range (150, 157):
  print(f'Video {i}')
  charfi_prediction_lecture_150_156.append(openpose_detect_fall_in_video('charfi', i))

Video 150
Video 151
Video 152
Video 153
==========Fall detected in frame 1092==========
Video 154
Video 155
Video 156


In [ ]:
print(charfi_prediction_lecture_150_156)

[0, 0, 0, 1, 0, 0, 0]


In [ ]:
charfi_prediction_lecture = charfi_prediction_lecture_130_139 + charfi_prediction_lecture_140_149 + charfi_prediction_lecture_150_156

In [ ]:
print(charfi_prediction_lecture)

[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]


## 4.4 Office

In [ ]:
charfi_prediction_office_157_159 = []
charfi_prediction_office_160_169 = []
charfi_prediction_office_170_179 = []
charfi_prediction_office_180_189 = []

In [ ]:
for i in range (157, 160):
  print(f'Video {i}')
  charfi_prediction_office_157_159.append(openpose_detect_fall_in_video('charfi', i))

Video 157
Video 158
==========Fall detected in frame 188==========
Video 159


In [ ]:
print(charfi_prediction_office_157_159)

[0, 1, 0]


In [ ]:
for i in range (160, 170):
  print(f'Video {i}')
  charfi_prediction_office_160_169.append(openpose_detect_fall_in_video('charfi', i))

Video 160
==========Fall detected in frame 310==========
Video 161
==========Fall detected in frame 152==========
Video 162
==========Fall detected in frame 400==========
Video 163
==========Fall detected in frame 129==========
Video 164
Video 165
==========Fall detected in frame 104==========
Video 166
==========Fall detected in frame 202==========
Video 167
Video 168
==========Fall detected in frame 234==========
Video 169
==========Fall detected in frame 678==========


In [ ]:
print(charfi_prediction_office_160_169)

[1, 1, 1, 1, 0, 1, 1, 0, 1, 1]


In [ ]:
for i in range (170, 180):
  print(f'Video {i}')
  charfi_prediction_office_170_179.append(openpose_detect_fall_in_video('charfi', i))

Video 170
==========Fall detected in frame 99==========
Video 171
Video 172
==========Fall detected in frame 258==========
Video 173
==========Fall detected in frame 138==========
Video 174
==========Fall detected in frame 94==========
Video 175
Video 176
Video 177
Video 178
Video 179


In [ ]:
print(charfi_prediction_office_170_179)

[1, 0, 1, 1, 1, 0, 0, 0, 0, 0]


In [ ]:
for i in range (180, 190):
  print(f'Video {i}')
  charfi_prediction_office_180_189.append(openpose_detect_fall_in_video('charfi', i))

Video 180
Video 181
==========Fall detected in frame 127==========
Video 182
Video 183
Video 184
Video 185
Video 186
Video 187
Video 188
==========Fall detected in frame 103==========
Video 189


In [ ]:
print(charfi_prediction_office_180_189)

[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]


In [ ]:
charfi_prediction_office = charfi_prediction_office_157_159 + charfi_prediction_office_160_169 + charfi_prediction_office_170_179 + charfi_prediction_office_180_189

In [ ]:
print(charfi_prediction_office)

[0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]


## 4.5 Office_2

In [ ]:
charfi_prediction_office_2_190_199 = []
charfi_prediction_office_2_200_209 = []
charfi_prediction_office_2_210_220 = []

In [ ]:
for i in range (190, 200):
  print(f'Video {i}')
  charfi_prediction_office_2_190_199.append(openpose_detect_fall_in_video('charfi', i))

Video 190
==========Fall detected in frame 807==========
Video 191
==========Fall detected in frame 199==========
Video 192
==========Fall detected in frame 862==========
Video 193
==========Fall detected in frame 251==========
Video 194
Video 195
Video 196
Video 197
Video 198
Video 199


In [ ]:
print(charfi_prediction_office_2_190_199)

[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]


In [ ]:
for i in range (200, 210):
  print(f'Video {i}')
  charfi_prediction_office_2_200_209.append(openpose_detect_fall_in_video('charfi', i))

Video 200
Video 201
==========Fall detected in frame 55==========
Video 202
Video 203
Video 204
==========Fall detected in frame 642==========
Video 205
==========Fall detected in frame 398==========
Video 206
Video 207
Video 208
==========Fall detected in frame 118==========
Video 209


In [ ]:
print(charfi_prediction_office_2_200_209)

[0, 1, 0, 0, 1, 1, 0, 0, 1, 0]


In [ ]:
for i in range (210, 221):
  print(f'Video {i}')
  charfi_prediction_office_2_210_220.append(openpose_detect_fall_in_video('charfi', i))

Video 210
==========Fall detected in frame 130==========
Video 211
Video 212
Video 213
Video 214
Video 215
Video 216
Video 217
Video 218
Video 219
Video 220


In [ ]:
print(charfi_prediction_office_2_210_220)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
charfi_prediction_office_2 = charfi_prediction_office_2_190_199 + charfi_prediction_office_2_200_209 + charfi_prediction_office_2_210_220

In [ ]:
print(charfi_prediction_office_2)

[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# 5 Evaluate

In [ ]:
ur_truth = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [ ]:
evaluate(ur_truth, ur_prediction)

True Positive: 24 
False Negative: 6 
False Positive: 23 
True Negative: 17 
Others: 0
Accuracy: 58.57% 
Sensitivity: 80.0% 
Specificity: 42.5%
